In [1]:
from MDP import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from math import floor

In [69]:
# Create the gridworld, and set cell values to equal rewards
print('(Creating Willemsen Gridworld)')
grid = np.ones((3, 9)) * -1
grid[1,:8] = 0
grid[1, 8] = 0.1
grid[2, :8] = 0
terminal_state = []
for i in [0,2]:
    for j in range(8):
        terminal_state.append([i,j])
terminal_state.append([1,8])
terminal_state = np.array(terminal_state)
initial_state = np.array([1, 0])
blacked_state = np.array([[0, 8],[2, 8]])

# fig, ax, im, cb = plot_gridworld(grid, terminal_state, initial_state, blacked_state, fig=None, ax=None, show=True)

(Creating Willemsen Gridworld)


In [70]:
n_episodes = 100
max_steps = 100
gamma = 0.8
alpha = 0.1

EXPLORATION_TYPE = 'GREEDY'

if EXPLORATION_TYPE == 'RANDOM':
    epsilons = np.ones(n_episodes)
elif EXPLORATION_TYPE == 'GREEDY':
    epsilons = np.flip(np.linspace(0.05, 1, n_episodes))

Learn = True
batch_size = 1
FILE_SIG = f"Results_n={n_episodes}, nB={batch_size}, exp={EXPLORATION_TYPE} GW2"

env = Gridworld(grid, terminal_state, initial_state, blacked_state, max_steps)

In [71]:
class global_context(Mask):
    def apply(self, transition):
        context = 0
        return context

class Noveltor(Agent):
    def combine_state_estimators(self, transition):
        # Epsilon Greey Action Selection based on Q values
        if np.random.random() > 1 - self.epsilon:
            action = self.actions[np.random.randint(self.n_actions)]
        else:
            Qs = self.estimators[1].evaluate(transition)
            max_value = max(Qs.values())
            max_actions = [k for k,v in Qs.items() if v == max_value]
            action = np.random.choice(max_actions)
        return action

In [72]:
agent = Noveltor(estimators = [Estimator(approximator=bQt_novel_alpha(gamma=0.8), mask=identity()), 
                           Estimator(approximator=bellman_N_table(alpha=0.1, gamma=0.8), mask=identity()),
                           Estimator(global_N_abstractor(alpha=0.1, gamma=0.8), mask=global_context()),
                           Estimator(global_Q_abstractor(gamma=0.8), mask=global_context())],
                buffer = ReplayMemory(max_size=10000),
                targets = [Target()],
                counter = state_table(),
                batch_size=batch_size)

In [81]:
env.reset()
agent.reset_trajectory()
agent.epsilon = epsilons[0]

In [82]:
action = agent.select_action(env.transition)

transition = env.step(action)

agent.observe(transition)

print(action, agent.estimators[1].approximator.table)

up defaultdict(<function table.__init__.<locals>.<lambda> at 0x0000016FD7FD0CA0>, {(1, 0): {'up': 0, 'down': 0.15933333333333335, 'left': 0, 'right': 0}, (2, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0}})


In [84]:
agent.trajectory

[T((1, 0),up)-->(0, 0))]